References and Borrowing
---

What is Ownership?
---

Ownership is a set of rules that govern how a Rust program manages memory.  
All programs have to manage the way they use a computer’s memory while running.  
Some languages have garbage collection that regularly looks for no-longer-used  
memory as the program runs; in other languages, the programmer must explicitly  
allocate and free the memory. Rust uses a third approach: memory is managed  
through a system of ownership with a set of rules that the compiler checks.  
If any of the rules are violated, the program won’t compile. None of the  
features of ownership will slow down your program while it’s running.

Because ownership is a new concept for many programmers, it does take some time  
to get used to. The good news is that the more experienced you become with Rust  
and the rules of the ownership system, the easier you’ll find it to naturally  
develop code that is safe and efficient. Keep at it!

When you understand ownership, you’ll have a solid foundation for understanding  
the features that make Rust unique. In this chapter, you’ll learn ownership by  
working through some examples that focus on a very common data structure:  

- strings.

### Stack and Heap
***

Many programming languages don’t require you to think about the stack and the  
heap very often. But in a systems programming language like Rust, whether a  
value is on the stack or the heap affects how the language behaves and why you  
have to make certain decisions. Parts of ownership will be described in relation  
to the stack and the heap later in this chapter, so here is a brief explanation  
in preparation.

Both the stack and the heap are parts of memory available to your code to use at  
runtime, but they are structured in different ways. The stack stores values in  
the order it gets them and removes the values in the opposite order. This is  
referred to as last in, first out. Think of a stack of plates: when you add more  
plates, you put them on top of the pile, and when you need a plate, you take one  
off the top. Adding or removing plates from the middle or bottom wouldn’t work  
as well! Adding data is called pushing onto the stack, and removing data is  
called popping off the stack. All **`data stored on the stack`** must have a known,  
**`fixed size`**. Data with an **`unknown size at compile time`** or a size that might  
change must be **`stored on the heap`** instead.

The heap is less organized: when you put data on the heap, you request a certain  
amount of space. The memory allocator finds an empty spot in the heap that is  
big enough, marks it as being in use, and returns a pointer, which is the  
address of that location. This process is called allocating on the heap and is  
sometimes abbreviated as just allocating (pushing values onto the stack is not  
 considered allocating). Because the **`pointer to the heap is a known, fixed size,`**  
 you can **`store the pointer on the stack`**, but when you want the actual data, you  
 must follow the pointer. Think of being seated at a restaurant. When you enter,  
 you state the number of people in your group, and the host finds an empty table  
 that fits everyone and leads you there. If someone in your group comes late,  
 they can ask where you’ve been seated to find you.

Pushing to the stack is faster than allocating on the heap because the allocator  
never has to search for a place to store new data; that location is always at  
the top of the stack. Comparatively, allocating space on the heap requires more    
work because the allocator must first find a big enough space to hold the data  
and then perform bookkeeping to prepare for the next allocation.

Accessing data in the heap is slower than accessing data on the stack because  
you have to follow a pointer to get there. Contemporary processors are faster  
if they jump around less in memory. Continuing the analogy, consider a server at  
a restaurant taking orders from many tables. It’s most efficient to get all the  
orders at one table before moving on to the next table. Taking an order from  
table A, then an order from table B, then one from A again, and then one from B  
again would be a much slower process. By the same token, a processor can do its  
job better if it works on data that’s close to other data (as it is on the stack)  
rather than farther away (as it can be on the heap).

When your code calls a function, the values passed into the function (including,  
potentially, pointers to data on the heap) and the function’s local variables  
get pushed onto the stack. When the function is over, those values get popped  
off the stack.

Keeping track of what parts of code are using what data on the heap, minimizing  
the amount of duplicate data on the heap, and cleaning up unused data on the  
heap so you don’t run out of space are all problems that ownership addresses.  
Once you understand ownership, you won’t need to think about the stack and the  
heap very often, but knowing that the main purpose of ownership is to manage  
heap data can help explain why it works the way it does.


Mutable References
---

In [4]:
{
    let mut s = String::from("hello"); // s comes into scope
    println!("{}", s);

    change(&mut s);
    println!("{}", s);
}

hello
hello, world


()

In [3]:
fn change(some_string: &mut String){
    some_string.push_str(", world");
}

In [5]:
{
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;
    println!("{}", r1);
    println!("{}", r2);
}

Error: cannot borrow `s` as mutable more than once at a time

This error says that this code is invalid because we **`cannot borrow s as mutable`**  
**`more than once at a time`**. The first mutable borrow is in r1 and must last until  
it’s used in the println!, but between the creation of that mutable reference  
and its usage, we tried to create another mutable reference in r2 that borrows  
the same data as r1.

The restriction preventing multiple mutable references to the same data at the  
same time allows for mutation but in a very controlled fashion. 

It’s something that new Rustaceans struggle with because most languages let you  
mutate whenever you’d like. The benefit of having this restriction is that Rust  
can prevent data races at compile time. A data race is similar to a race  
condition and happens when these three behaviors occur:

- Two or more pointers access the same data at the same time.
- At least one of the pointers is being used to write to the data.
- There’s no mechanism being used to synchronize access to the data.

Data races cause undefined behavior and can be difficult to diagnose and fix  
when you’re trying to track them down at runtime; Rust prevents this problem by  
refusing to compile code with data races!

As always, we can use curly brackets to create a new scope, allowing for   
multiple mutable references, just not simultaneous ones:


In [7]:
{
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
        println!("borrowing [s] mutable in [inner] scope {}", r1);
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
    println!("borrowing [s] mutable in [outer] scope {}", r2);
}
// In actual RUST env ^ this could be outer scope
// BUT in Jupyter all rust code is in static scope so we needed to wrap it in 
// another BLOCK

borrowing [s] mutable in [inner] scope hello
borrowing [s] mutable in [outer] scope hello


Rust enforces a similar rule for combining mutable and immutable references.  
This code results in an error:

In [10]:
{
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    // let r3 = &s; // no problem
    let r3 = &mut s; // BIG PROBLEM

    println!("{}, {}, and {}", r1, r2, r3);
}

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

Whew! We also `cannot have a mutable reference while we have an immutable one` to  
the same value.

Users of an `immutable reference don’t expect the value to suddenly change` out  
from under them! However, multiple immutable references are allowed because no  
one who is just reading the data has the ability to affect anyone else’s  
reading of the data.

Note that a reference’s scope starts from where it is introduced and continues  
through the last time that reference is used. For instance, this code will  
compile because the last usage of the immutable references, the println!, occurs  
before the mutable reference is introduced:

In [11]:
{
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem

    println!("{} and {}", r1, r2);
    // variables r1 and r2 will not be used after this point (code span)

    let r3 = &mut s; // no problem, because this is non overlapping code span
    println!("{}", r3);
}

hello and hello
hello


()

The scopes of the immutable references r1 and r2 end after the println! where  
they are last used, which is before the mutable reference r3 is created.  
These scopes don’t overlap, so this code is allowed: the compiler can tell that  
the reference is no longer being used at a point before the end of the scope.

Even though borrowing errors may be frustrating at times, remember that it’s the  
Rust compiler pointing out a potential bug early (at compile time rather than at  
runtime) and showing you exactly where the problem is. Then you don’t have to  
track down why your data isn’t what you thought it was.

Dangling References
---